In [1]:
import numpy as np

# Import data

X_train = np.random.rand(7200,1728)
X_test = np.random.rand(1800,1728)
y_train = np.mean(X_train,axis=0)
y_test = np.mean(X_test,axis=0)

print(np.shape(X_train),np.shape(X_test),np.shape(y_train),np.shape(y_test))

(7200, 1728) (1800, 1728) (1728,) (1728,)


In [2]:
y_train=np.reshape(y_train,(-1,1))
y_test=np.reshape(y_test,(-1,1))

In [3]:
# Scale data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
'''
scaler.fit(y_train)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)
# Build X and y
X_train = data_train[:, 1:]
y_train = data_train[:, 0]
X_test = data_test[:, 1:]
y_test = data_test[:, 0]'''

'\nscaler.fit(y_train)\ny_train = scaler.transform(y_train)\ny_test = scaler.transform(y_test)\n# Build X and y\nX_train = data_train[:, 1:]\ny_train = data_train[:, 0]\nX_test = data_test[:, 1:]\ny_test = data_test[:, 0]'

In [4]:
# Import TensorFlow
import tensorflow as tf

# Model architecture parameters
n_input = 1728
n_neurons_1 = 2048
n_neurons_2 = 1024
n_neurons_3 = 512
n_neurons_4 = 256
n_target = 1

# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, n_input])
Y = tf.placeholder(dtype=tf.float32, shape=[None,1])

C:\Users\Skinny\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

In [6]:
# Layer 1: Variables for hidden weights and biases
W_hidden_1 = tf.Variable(weight_initializer([n_input, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
# Layer 2: Variables for hidden weights and biases
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
# Layer 3: Variables for hidden weights and biases
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
# Layer 4: Variables for hidden weights and biases
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output layer: Variables for output weights and biases
W_out = tf.Variable(weight_initializer([n_neurons_4, n_target]))  #notice the change
bias_out = tf.Variable(bias_initializer([n_target]))

In [7]:
# Hidden layer
hidden_1 = tf.nn.leaky_relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.leaky_relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.leaky_relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.leaky_relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (must be transposed)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out)) #notice the change

In [8]:
# Cost function
#mse = tf.losses.log_loss(np.abs(Y)+0.001, np.abs(out)+0.001)
mse = tf.reduce_mean(tf.squared_difference(out, Y))

In [9]:
# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

In [12]:
# Make Session
net = tf.Session()
# Run initializer
net.run(tf.global_variables_initializer())

# Number of epochs and batch size
epochs = 20
batch_size = 256

for e in range(epochs):
    #print(e,end=' ')
    # Shuffle training data
    shuffle_indices = np.random.permutation(np.arange(len(y_train)))
    X_train = X_train[shuffle_indices]
    y_train = y_train[shuffle_indices]

    # Minibatch training
    for i in range(0, len(y_train) // batch_size):
        start = i * batch_size
        batch_x = X_train[start:start + batch_size]
        batch_y = y_train[start:start + batch_size]
        # Run optimizer with batch
        net.run(opt, feed_dict={X: batch_x, Y: batch_y})

        # Show progress
        if np.mod(i, 5) == 0:
            # Prediction
            pred = net.run(out, feed_dict={X: X_test})
            
    # Print final MSE after Training
    mse_train = net.run(mse, feed_dict={X: X_train, Y: y_train})
    mse_test = net.run(mse, feed_dict={X: X_test, Y: y_test})
    print(mse_train, mse_test)
    

                        [[23.062513 22.798088 22.420292 ... 22.627428 22.308397 22.701813]]
                        [[-0.30637592 -0.2823919  -0.3910341  ... -0.24872117 -0.31222188
  -0.3468105 ]]
0.6714537 0.6668617
                        [[3.0993645 3.0624356 3.0132844 ... 3.1530666 3.0653243 2.9275928]]
                        [[-0.4914814  -0.6083073  -0.4791325  ... -0.60264975 -0.5176001
  -0.594613  ]]
1.122068 1.117864
                        [[1.9195507 1.7625601 1.8777034 ... 1.8284391 1.8865442 1.8018926]]
                        [[-0.14799127 -0.21863243 -0.12596866 ... -0.193755   -0.12720105
  -0.18538865]]
0.45661467 0.45810044
                        [[1.5800327 1.4481585 1.5432491 ... 1.5148073 1.5578972 1.4839578]]
                        [[0.39920276 0.3117935  0.3949527  ... 0.34726077 0.41339058 0.35105044]]
0.019134114 0.01971825
                        [[1.2873744 1.1722871 1.2634706 ... 1.2126741 1.2824713 1.2117794]]
                        [[1.2957547 1.1782